In [ ]:
!pip install lightkurve 
!pip install transitleastsquares
!pip install wotan
!pip install astroquery
import lightkurve as lk
import transitleastsquares as tls
from wotan import flatten, t14
import astroquery.mast as astromast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 247 kB 4.5 MB/s 
     |████████████████████████████████| 98 kB 6.8 MB/s 
     |████████████████████████████████| 41 kB 193 kB/s 
     |████████████████████████████████| 4.5 MB 35.7 MB/s 
     |████████████████████████████████| 832 kB 48.9 MB/s 
     |████████████████████████████████| 48 kB 5.4 MB/s 
     |████████████████████████████████| 4.0 MB 34.5 MB/s 
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11375 sha256=51ba1231b23b0e8b19da62319fb9af8c92a8fbd5f6d26d34358c8023173a56c3
  Stored in directory: /root/.cache/pip/wheels/93/08/0c/1b9866c35c8d3f136d100dfe88036a32e0795437daca089f70
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50467 sha256=634854a45fa6799e82a29eebb9739b71e9bbb87edebe30a7b97967474679cb16
  Stored in directory: /root/.cache/pip/wheels/38/f7/65/161985e7311dd484a23b3a5c9149995db

In [ ]:
import numpy as np
import pandas as pd
import scipy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from astropy.io import fits
import os 
import subprocess
import matplotlib.pyplot as plt

In [ ]:
import time
import tqdm
from astropy.stats import sigma_clip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
def load_file(n):
  return '/content/drive/Othercomputers/My Laptop/ML Project/MAST/tic2file-sec'+ str(n) +'.csv'

def read_csv(n):
  csv_file = load_file(n)
  df = pd.read_csv(csv_file)
  return df

# print(read_csv(21))

def light_curve(n,id):
  light_curve = lk.search_lightcurve('TIC' + str(id), exptime = 120, sector = n).download()
  return light_curve

# light_curve(21, read_csv(21)['tic_id'][0])

In [60]:
def load_lightcurve(n, tess_id):
  curves = []
  initial_path = '/content/drive/Othercomputers/My Laptop/ML Project/MAST/Sector ' + str(n) + '/'
  files = read_csv(n)['Filename'][read_csv.loc(read_csv['tic_id'] == tess_id)]
  for i in files:
    filepath = os.path.join(initial_path, i)
    curves.append(lk.search.open(filepath))
  light_curves = lk.LightCurveFileCollection(curves)
  return light_curves.stitch()

In [ ]:
read_csv(24).head()

,tic_id,Sector,Filename
0,275634677,24,tess2020106103520-s0024-0000000275634677-0180-...
1,428740087,24,tess2020106103520-s0024-0000000428740087-0180-...
2,441802949,24,tess2020106103520-s0024-0000000441802949-0180-...
3,188768068,24,tess2020106103520-s0024-0000000188768068-0180-...
4,343410928,24,tess2020106103520-s0024-0000000343410928-0180-...


### 2

In [59]:
# Takes some time to compute
def fits_data(n):
  df = read_csv(n)
  initial_path = '/content/drive/Othercomputers/My Laptop/ML Project/MAST/Sector ' + str(n) + '/'
  for i in range(len(df)):
    print(i)
    file_fit = os.path.join(initial_path, df['Filename'][i])
    if not os.path.exists(file_fit):
      continue
                
    try:
      tic = T0= depth= period =period_uncertainty=duration=t_mag=t_eff=radius_0=distinct_transit_count=a=b=log=rp_rs=depth_even=depth_odd=odd_even_mismatch=snr=SDE = []
      with fits.open(file_fit) as hdu:
        hdu.info()
        tic_id = hdu[0].header['TICID']
        tess_mag = hdu[0].header["TESSMAG"] # [mag]
        temp_eff = hdu[0].header["TEFF"] # K
        logg = hdu[0].header["LOGG"] # [cm/s2] log10 surface gravity
        rad = hdu[0].header['RADIUS'] # [solar radii] stellar radius 
        time = hdu[1].data['TIME']
        flux = hdu[1].data['PDCSAP_FLUX']  
        time, flux = tls.cleaned_array(time, flux)  
        flux = flux / np.median(flux)
        ab, mass, mass_min, mass_max, radius, radius_min, radius_max = tls.catalog_info(TIC_ID= tic_id)
        print('Searching with limb-darkening estimates using quadratic LD (a,b)=', ab)

        # print('Period', format(results.period, '.5f'), 'd at T0=', results.T0)
        # print(len(results.transit_times), 'transit times in time series:', ['{0:0.5f}'.format(i) for i in results.transit_times])
        # print('Number of data points during each unique transit', results.per_transit_count)
        # print('The number of transits with intransit data points', results.distinct_transit_count)
        # print('The number of transits with no intransit data points', results.empty_transit_count)
        # print('Transit depth', format(results.depth, '.5f'), '(at the transit bottom)')
        # print('Transit duration (days)', format(results.duration, '.5f'))
        # print('Transit depths (mean)', results.transit_depths)
        # print('Transit depth uncertainties', results.transit_depths_uncertainties)
        if np.isnan(mass) or np.isnan(radius):
            flatten_lc, trend_lc = flatten(time, flux, window_length=0.5, method='biweight', return_trend=True)
            flux = sigma_clip(flatten_lc, sigma_upper=3, sigma_lower=20)
            
            # if numpy.mean(flux) > 1.01 or numpy.mean(flux) < 0.99:
            #     # Normalize by the mean if needed
            #     flux = flux / numpy.mean(flux)
            model = tls.transitleastsquares(time, flux)
            results = model.power(u=ab,oversampling_factor=5)
        else:
            period_2 = 13.5
            tdur = t14(R_s=radius, M_s=mass, P=period_2, small_planet=False)
            flatten_lc, trend_lc = flatten(time, flux, window_length=3*tdur, method='biweight', return_trend=True)
            flux = sigma_clip(flatten_lc, sigma_upper=3, sigma_lower=20)
            # if numpy.mean(flux) > 1.01 or numpy.mean(flux) < 0.99:
            #     # Normalize by the mean if needed
            #     flux = flux / numpy.mean(flux)
            model = tls.transitleastsquares(time, flux)
            mstar_min = mass - 3 * mass_min
            if mstar_min < 1e-3:
                mstar_min = 0.0
            rstar_min = radius - 3 * radius_min
            if rstar_min < 1e-3:
                rstar_min = 0.0
            results = model.power(u=ab, M_star_min=mstar_min, M_star=mass, M_star_max=mass+3*mass_max, R_star_min=rstar_min, R_star=radius, R_star_max=radius+3*radius_max, 
                            oversampling_factor=5)
        tic.append(tic_id)
        T0.append(results.T0) 
        depth.append(results.depth) 
        period.append(results.period)
        period_uncertainty.append(results.period_uncertainty)
        duration.append(results.duration * 24)
        t_mag.append(tess_mag) 
        t_eff.append(temp_eff)
        radius_0.append(rad)
        distinct_transit_count.append(results.distinct_transit_count)
        a.append(ab[0])
        b.append(ab[1])
        log.append(logg)
        rp_rs.append(results.rp_rs)
        depth_even.append(results.depth_mean_even[0])
        depth_odd.append(results.depth_mean_odd[0])
        odd_even_mismatch.append(results.odd_even_mismatch)
        snr.append(results.snr)
        SDE.append(results.SDE)
    except (OSError, TypeError):
        return None
  new_dataframe = pd.DataFrame({'tic_id': tic, 'T0': T0, 'depth': depth, 'period' : period, 'period_uncertainty':period_uncertainty,'duration':duration, 'tess_mag': t_mag,
                                'temp_eff': t_eff, 'radius': radius_0, 'distinct_transit_count': distinct_transit_count, 'a': a, 'b': b, 'logg': log, 'rp_rs': rp_rs, 'depth_even':depth_even,
                                'depth_odd':depth_odd, 'odd_even_mismatch':odd_even_mismatch, 'snr': snr, 'sde': SDE})
  return (new_dataframe,results)


In [37]:
# fits_data(23)[0]['tic_id']

https://github.com/hippke/tls/blob/master/tutorials/11%20A%20planet%20from%20TESS%20with%20TLS.ipynb 

https://github.com/hippke/wotan/blob/master/tutorials/01%20Basic%20functionality.ipynb

Wotan documentation- https://wotan.readthedocs.io/_/downloads/en/stable/pdf/

In [ ]:
def stellar_params(n):

  file_csv = fits_data(n)[0]
  tic = file_csv['tic_id']
  mass = []
  dist = []
  logg = []
  lum = []
  rho =[]
  for id in tic:
    # print(id)
    result = astromast.Catalogs.query_criteria(catalog='TIC', ID=tic)
    # print(result['mass'])
    mass.append(result['mass'])
    # print(id)
    dist.append(result['d'])
    logg.append(result['logg'])
    lum.append(result['lum'])
    rho.append(result['rho'])

    #df['logg'] = logg
  file_csv['Mass'] = mass
  file_csv['distance'] = dist
  file_csv['lum'] = lum
  file_csv['rho'] = rho
  return file_csv


In [ ]:
def params(n):
  tic_file = stellar_params(n)
  tic =  tic_file['tic_id']
  camera = ccd = ra = dec= pmra= pmdec= pmtotal= mh = []
  count = 0
  for id in tic:
    initial_path = '/content/drive/Othercomputers/My Laptop/ML Project/MAST/Sector ' + str(n) + '/'
    index = tic_file.loc(tic_file['tic_id'] == id)
    filename = os.path.join(initial_path, tic_file['Filename'][index])
    hdu = fits.open(filename)
    camera.appen(hdu[0].header['CAMERA'])
    ccd.append(hdu[0].header['CCD'])
    ra.append(float(hdu[0].header['RA_OBJ']))
    dec.append(float(hdu[0].header['DEC_OBJ']))
    pmra.append(float(hdu[0].header['PMRA']))
    pmdec.append(float(hdu[0].header['PMDEC']))
    pmtotal.append(float(hdu[0].header['PMTOTAL']))
    mh.append(float(hdu[0].header['MH']))

  tic_file['Camera'] = camera
  tic_file['CCD'] = ccd
  tic_file['RA'] = ra
  tic_file['DEC'] = dec
  tic_file['PMRA'] = pmra
  tic_file['PMDEC'] = pmdec
  tic_file['PMTOTAL'] = pmtotal
  tic_file['MH'] = mh

  return tic_file

In [ ]:
def plot_phase(result,periodogram = False):
    if perdiodogram == True:
      plt.figure()
      ax = plt.gca()
      ax.axvline(results.period, alpha=0.4, lw=3)
      plt.xlim(numpy.min(results.periods), numpy.max(results.periods))
      for n in range(2, 10):
          ax.axvline(n*results.period, alpha=0.4, lw=1, linestyle="dashed")
          ax.axvline(results.period / n, alpha=0.4, lw=1, linestyle="dashed")
      plt.ylabel(r'SDE')
      plt.xlabel('Period (days)')
      plt.plot(results.periods, results.power, color='black', lw=0.5)
      plt.xlim(0, max(results.periods));  
    else:
      plt.figure()
      plt.plot(result.model_folded_phase, result.model_folded_model, color='red')
      plt.scatter(result.folded_phase, result.folded_y, color='blue', s=10, alpha=0.5, zorder=2)
      plt.xlim(0.48, 0.52)
      plt.xlabel('Phase')
      plt.ylabel('Relative flux');

### 3

https://heasarc.gsfc.nasa.gov/docs/tess/LightCurve-object-Tutorial.html

In [67]:
def folded_light_curve(n, tic_id,T0,period):
  data = params(n)
  tess_id = data.loc(data['tic_id'] == tic_id)
  lightcurve = load_lightcurve(n,tess_id)
  duration = data['duration'][tess_id]
  cleaned_lightcurve = lightcurve.remove_outliers(sigma=20, sigma_upper=5)
  fold = cleaned_lightcurve.fold(period, t0=T0)
  norm_duration = (duration_hours / 24.0) / period
  mask = np.abs(fold.phase) < (norm_duration * 1.5)
  mask1 = np.in1d(cleaned_lightcurve.time, fold.time_original[mask])
  x, y = cleaned_lightcurve.flatten(return_trend=True, mask=mask1)
  return x.fold(period, t0=T0)

def phase_fold_lk(n, tic_id,T0,period):
  tess_id = data.loc(data['tic_id'] == tic_id)
  lc = folded_lightcurve(n, tic_id, T0,period)
  duration = params['duration'][tess_id]

  norm_duration = (duration / 24.0) / period
  mask = np.abs(lc.phase) < (0.5*norm_duration)
  zoomed = lc[mask]
  global_view = zoomed.bin(n_bins=201) 
  global_view = 2.0 * (global_view / np.abs(global_view.flux.min()))
  global_view = global_view.remove_nans()

  mask_2 = np.abs(lc.phase) < (2.0*norm_duration)
  zoom_lk = lc[mask_2]
  local_lk = zoom_lk.bin(n_bins=81) 
  local_lk = 2.0 * (local_lk / np.abs(local_lk.flux.min()))
  local_lk = local_lk.remove_nans()

  lc_2 = folded_lightcurve(n, tic_id, period, T0 + 0.25 * period)
  half_phase = lc_2.bin(n_bins=201) 
  half_phase = 2.0 * (half_phase / np.abs(half_phase.flux.min()))
  half_phase = half_phase.remove_nans()

  half_even = zoom_lk.bin(n_bins=81)  

  lc_3 = folded_lightcurve(n, tic_id, period, t0 - 0.5 * period)
  mask_3 = np.abs(lc_3.phase) < (2.0*norm_duration)
  zoom_lc = lc_3[mask_3]
  half_odd = zoom_lc.bin(n_bins=81) 

  time_net = np.concatenate((half_odd.time, half_even.time))
  flux_net = np.concatenate((half_odd.flux, half_even.flux))
  concatenated = lk.LightCurve(time_net, flux_net)
  normed = 2.0 * (concatenated / np.abs(concatenated.flux.min()))
                
  return global_view, local_lk, half_phase, normed

def final_lk(n, tic_id):
  data = params(n)
  try:
    period = data['period'][tess_id]
    T0 = data['T0'][tess_id]
    return phase_fold_lk(n, tic_id,T0,period)
  except:
    return None, None, None, None

In [ ]:
def cnn_model(global_shape,local_shape,half_phase, stellar_transit):

  def nn(x, filter, kernel_size,act):
    return layers.Conv1D(filters, kernel_size,padding = 'same',activation = act)(x)

  def global_view_model(x):
    for i in [16,32,64,128,256]:
      x = nn(x,i,7,'relu')
      x = nn(x,i,7,'relu')
      x = layers.MaxPooling1D(5, 2)(x)
    return layers.Flatten()(x)

  def local_view_model(x):
    for i in [16,32]:
      x = nn(x,i,7,'relu')
      x = nn(x,i,7,'relu')
      x = layers.MaxPool1D(5,2)(x)
    return layers.Flatten()(x)

  global_in = tf.keras.Input(shape=global_shape)
  global_model = global_view_model(global_in)
  local_in = Input(shape=local_shape)
  local_model = local_view_model(local_in)
  half_phase_in = Input(shape=half_phase)
  half_phase_model = local_view_model(half_phase_in)

  combine = layers.Concacatenate(axis=1)([global_model,local_model, half_phase_model])

  stellar_transit_in =[]
  for i in stellar_transit:
    y = Input(shape=(1,))
    combine = layers.Concacatenate(axis=1)([combine, y])
    stellar_transit_in.append(y)
  # Fully connected layers
  model = keras.models.Sequential()
  model.add(layers.Dense(512,activation='relu'))
  for i in range(3):
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(512,activation='relu'))
  # Activation for output layer is sigmoid
  model.add(layers.Dense(1,activation='sigmoid',bias_initializer = keras.initializers.Constant(value=0)))
  out = model(combine)

  final_model = keras.Model(inputs= [global_in,local_in, half_phase_in, stellar_transit_in], outputs = [out])
  adam = keras.optimizers.Adam(learning_rate=1e-5, epsilon= 1e-8)
  final_model.compile(loss= keras.losses.BinaryCrossentropy() , optimizer = adam, metrics = [keras.metrics.BinaryAccuracy()])
  
  return final_model



